# Практическая работа №6. Атака по переносу (Transfer attack) на модели ИИ

Выполнил Сердюков Матвей, ББМО-01-23

## Цель задания

Изучить концепцию атаки по переносу, где противоречивые примеры, созданные для одной модели,
используются для атаки на другую модель. Это задание требует создания нескольких моделей,
генерации противоречивых примеров для одной модели и проверки их на другой модели.

## Задачи

1. Загрузить несколько моделей, обученных на датасете MNIST.
2. Изучить теоретические основы атаки по переносу.
3. Реализовать атаку FGSM на одну модель и проверить, как противоречивые примеры влияют на
другую модель.
4. Оценить точность обеих моделей на противоречивых примерах и проанализировать
переносимость атак.

## Загрузка и создание двух различных моделей

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Компиляция модели
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model1.fit(train_images, train_labels, epochs=5)

# Сохранение модели
model1.save('mnist_model1.h5')

# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)

# Сохранение модели
model2.save('mnist_model2.h5')

2024-12-20 19:17:17.093989: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 19:17:17.262392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-20 19:17:17.328913: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-20 19:17:17.345899: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 19:17:17.467226: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8775 - loss: 0.4353
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9668 - loss: 0.1155
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9782 - loss: 0.0748
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9829 - loss: 0.0574
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9883 - loss: 0.0419


Epoch 1/5


/home/baksist/.virtualenvs/aszii/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9078 - loss: 0.3143
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9846 - loss: 0.0528
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9912 - loss: 0.0285
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9942 - loss: 0.0183
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9960 - loss: 0.0138


## Реализация атаки FGSM на первую модель

In [6]:
import numpy as np
# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    # Применение знака градиента к изображению
    perturbed_image = image + epsilon * np.sign(gradient)
    perturbed_image = np.clip(perturbed_image, 0, 1) # Убедиться, что значения остаются в пределах [0, 1]
    return perturbed_image
# Генерация противоречивых примеров для первой модели
def generate_fgsm_adversarial(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        #image = images[i].reshape(1, 28, 28, 1)
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)))
        #label = labels[i]
        label = tf.convert_to_tensor(labels[i])
        # Вычисление градиента
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
        gradient = tape.gradient(loss, image)
        adv_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adv_image.reshape(28, 28))
    return np.array(adversarial_images)
# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_fgsm_adversarial(model1, test_images, test_labels, epsilon)

## Оценка противоречивых примеров на обеих моделях

In [7]:
# Оценка первой модели на противоречивых примерах
test_labels_argmax = np.argmax(test_labels, axis=1) # Преобразование one-hot меток в целые числа
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.0989 - loss: 6.2114
Accuracy of model1 on adversarial examples: 0.12520000338554382
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9608 - loss: 0.1441
Accuracy of model2 on adversarial examples from model1: 0.9652000069618225


## Анализ переносимости атак

In [8]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_fgsm_adversarial(model2, test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.9494 - loss: 0.1592
Accuracy of model1 on adversarial examples from model2: 0.9549999833106995


## Вывод

Осуществив FGSM-атаку на первую модель, сгенерированные примеры также смогли повлиять на вторую модель, однако, в значительно меньшей степени. Если для первой модели точность на противоречивых примерах составила 9,9%, то у второй модели точность упала лишь до 95%. Аналогичное поведение можно наблюдать и в обратной ситуации, применив противоречивые примеры второй модели для первой.

Таким образом, атака по переносу позволяет использовать одни и те же противоречивые примеры для разных моделей, но с разной степенью эффективности.